# 第13章 RAG

## 13.3 RAG 向けに LLM を指示チューニングする

### 13.3.2 指示チューニングしたモデルを LangChain で使う

#### 環境の準備

In [ ]:
!pip install transformers[torch,sentencepiece] langchain langchain-community langchain-huggingface faiss-cpu jq

In [ ]:
from huggingface_hub import notebook_login

# Hugging Face Hubにログイン
notebook_login()

In [ ]:
from transformers.trainer_utils import set_seed

# 乱数のシードを設定
set_seed(42)

#### Chat Model の作成

In [ ]:
import torch
from langchain_huggingface import (
    ChatHuggingFace,
    HuggingFacePipeline,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)

# Hugging Face Hubにおけるモデル名を指定
model_name = "llm-book/Swallow-7b-hf-oasst1-21k-ja-aio-retriever"

# モデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# トークナイザを読み込む
tokenizer = AutoTokenizer.from_pretrained(model_name)

# テキスト生成用のパラメータを指定
generation_config = {
    "max_new_tokens": 32,
    "do_sample": False,
    "temperature": None,
    "top_p": None,
}

# テキスト生成を行うパイプラインを作成
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    **generation_config,
)

# パイプラインからLangChainのLLMコンポーネントを作成
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# LLMコンポーネントを元にChat Modelコンポーネントを作成
chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer)

#### Embedding Model の作成

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Hugging Face Hubにおけるモデル名を指定
embedding_model_name = "BAAI/bge-m3"

# モデル名からEmbedding Modelを初期化
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"model_kwargs": {"torch_dtype": torch.float16}},
)

#### データストアの構築

In [ ]:
# 検索対象の文書集合のファイルをダウンロード
!wget  \
https://github.com/ghmagazine/llm-book/raw/main/chapter13/docs.json

In [ ]:
from langchain_community.document_loaders import JSONLoader

# JSONファイルから文書を読み込むためのDocument Loaderを初期化
document_loader = JSONLoader(
    file_path="./docs.json",  # 読み込みを行うファイル
    jq_schema=".text",  # 読み込み対象のフィールド
    json_lines=True,  # JSON Lines形式のファイルであることを指定
)

# 文書の読み込みを実行
documents = document_loader.load()

# 読み込まれた文書数を確認
print(len(documents))

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 文書を指定した文字数で分割するText Splitterを初期化
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # 分割する最大文字数
    chunk_overlap=100,  # 分割された文書間で重複させる最大文字数
    add_start_index=True,  # 元の文書における開始位置の情報を付与
)

# 文書の分割を実行
split_documents = text_splitter.split_documents(documents)

# 分割後の文書数を確認
print(len(split_documents))

#### 検索対象の文書のベクトルインデックスの作成

In [ ]:
from langchain_community.vectorstores import FAISS

# 分割後の文書と文埋め込みモデルを用いて、Faissのベクトルインデックスを作成
vectorstore = FAISS.from_documents(split_documents, embedding_model)

# ベクトルインデックスに登録された文書数を確認
print(vectorstore.index.ntotal)

#### Retriever コンポーネントの作成

In [ ]:
# ベクトルインデックスを元に文書の検索を行うRetrieverを初期化
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
from pprint import pprint

# 文書の検索を実行
retrieved_documents = retriever.invoke("四国地方で一番高い山は？")

# 検索された文書を確認
pprint(retrieved_documents)

#### RAG の Chain の構築と実行

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 任意のqueryからメッセージを構築するPrompt Templateを作成
rag_prompt_text = (
    "あなたには今からクイズに答えてもらいます。"
    "問題を与えますので、その解答のみを簡潔に出力してください。\n"
    "また解答の参考になりうるテキストを与えます。"
    "解答を含まない場合もあるのでその場合は無視してください。\n\n"
    "---\n{context}\n---\n\n問題: {query}"
)
rag_prompt_template = ChatPromptTemplate.from_messages(
    [("user", rag_prompt_text)]
)

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

def format_documents_func(documents: list[Document]) -> str:
    """文書のリストを改行で連結した一つの文字列として返す"""
    return "\n\n".join(
        document.page_content for document in documents
    )

# 定義した関数の処理を行うRunnableを作成
format_documents = RunnableLambda(format_documents_func)

In [ ]:
from langchain_core.prompt_values import ChatPromptValue

def chat_model_resp_only_func(
    chat_prompt_value: ChatPromptValue,
) -> str:
    """chat_modelにchat_prompt_valueを入力し、
    出力からモデルの応答部分のみを文字列で返す"""
    chat_prompt = chat_model._to_chat_prompt(
        chat_prompt_value.messages
    )
    chat_output_message = chat_model.invoke(chat_prompt_value)
    response_text = chat_output_message.content[len(chat_prompt) :]
    return response_text

# 定義した関数の処理を行うRunnableを作成
chat_model_resp_only = RunnableLambda(chat_model_resp_only_func)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# RAGの一連の処理を行うChainを作成
rag_chain = (
    {
        "context": retriever | format_documents,
        "query": RunnablePassthrough(),
    }
    | rag_prompt_template
    | chat_model_resp_only
)

In [ ]:
# Chainを実行し、結果を確認
rag_chain_output = rag_chain.invoke("四国地方で一番高い山は？")
print(rag_chain_output)